In [0]:
-- For each customer, list their orders in chronological order and compute:

-- - a sequence number for each order for that customer
-- - the number of days since their previous order (null for the first order)

-- **Output columns**

-- - `customer_id`
-- - `order_id`
-- - `order_ts`
-- - `order_number_for_customer`
-- - `days_since_prev_order`

with multiple_orders as (SELECT c.customer_unique_id as cust_id
FROM olist_customers_dataset c 
JOIN olist_orders_dataset o ON c.customer_id = o.customer_id
GROUP BY c.customer_unique_id
HAVING count(o.order_id) > 1)

SELECT c.customer_unique_id as cust_id,
o.order_id as order_id,
o.order_purchase_timestamp::date as order_ts,
ROW_NUMBER() OVER(PARTITION BY c.customer_unique_id ORDER BY o.order_purchase_timestamp) as order_number_for_customer,
DATE_PART('day', o.order_purchase_timestamp - LAG(o.order_purchase_timestamp) OVER(PARTITION BY c.customer_unique_id ORDER BY o.order_purchase_timestamp)) as days_since_prev_order
FROM olist_orders_dataset o
JOIN olist_customers_dataset c ON c.customer_id = o.customer_id
JOIN multiple_orders m ON m.cust_id = c.customer_unique_id
ORDER BY cust_id, order_number_for_customer





























-- WITH customer_order_counts AS (
--     SELECT
--         c.customer_unique_id,
--         COUNT(odd.order_id) AS total_orders
--     FROM
--         olist_orders_dataset odd
--     JOIN olist_customers_dataset c
--         ON c.customer_id = odd.customer_id
--     GROUP BY
--         c.customer_unique_id
--     HAVING COUNT(odd.order_id) > 1
-- )

-- SELECT 
--     c.customer_unique_id AS cust_id, 
--     odd.order_id AS order_id,
--     odd.order_purchase_timestamp::date AS order_ts,
--     ROW_NUMBER() OVER (PARTITION BY c.customer_unique_id ORDER BY odd.order_purchase_timestamp) AS order_number_for_customer,
--     DATE_PART('day', odd.order_purchase_timestamp - 
--         LAG(odd.order_purchase_timestamp) OVER (PARTITION BY c.customer_unique_id ORDER BY odd.order_purchase_timestamp)) AS days_since_prev_order
-- FROM 
--     olist_orders_dataset odd
-- JOIN 
--     olist_customers_dataset c 
--     ON c.customer_id = odd.customer_id
-- JOIN 
--     customer_order_counts coc
--     ON coc.customer_unique_id = c.customer_unique_id
-- ORDER BY 
--     c.customer_unique_id, odd.order_purchase_timestamp;
